This script runs EC821B HW03 Pt1: simulation of a multivariate normal probability.

$\Pr(Y\ge 0),\ Y\in \mathcal{R}^d,\ Y\sim \mathsf{N}(\mu, \Omega)$

First, I simulate the probability using a GHK Simulator.
Then, I simulate using a Crude Frequency and a Crude Frequency Antivariate simulator.

In [1]:
# Set Working Directory
cd("/Volumes/HDD_lxw/Users/LukeXWatson/Dropbox/02_CLASSES/2017_02_Spring/821B_Cross Section/HW/HW3")
# Get Standard Packages Running
using Distributions, Plots, DataFrames, StatsFuns, Optim

In [2]:
# Set Primatives of the Model
mu = [0.5 -0.25 0.5 -0.25 0.5];
omg = [1.0 0.3 0.3 0.2 -0.1; 0.3 1.0 0.3 0.2 -0.1; 
    0.3 0.3 1.0 0.2 -0.1; 0.2 0.2 0.2 1.0 -0.1; -0.1 -0.1 -0.1 -0.1 1];
d = length(mu);

In [3]:
# We want the lower triangle of the cholesky decomposition of Omega.
G = chol(omg)'

5×5 LowerTriangular{Float64,Array{Float64,2}}:
  1.0    ⋅           ⋅           ⋅          ⋅      
  0.3   0.953939     ⋅           ⋅          ⋅      
  0.3   0.22014     0.928191     ⋅          ⋅      
  0.2   0.14676     0.116024    0.961769    ⋅      
 -0.1  -0.0733799  -0.0580119  -0.0649844  0.988447

In [4]:
function GHK_Sim(R::Int64)
    # This function is a non-general GHK simulator for the above probability.
    # This function takes only the number of simulations as the argument.
    # Random Seed
    srand(757537);
    # Radnom Draws
    # Containers
    xi = zeros(d,R);
    nxi = zeros(d,R);    
    eta = zeros(d,R);
    nu = zeros(d,R);
    P = zeros(d,R);
    x1 = cdf(Normal(),((-1*mu[1]) / G[1,1]))
    eta[1,:] = rand(Uniform(x1,1), R)
    nu[1,:] = norminvcdf.(eta[1,:])
    P[1,:] =  (1 - x1)*ones(1,R)
    for r = 1:R
        for j = 2:d
            xi[j,r] = ((-1)*(mu[j] + (G[j,1:(j-1)]'*nu[1:(j-1),r])) / G[j,j])[1,1]
            nxi[j,r] = cdf(Normal(), xi[j,r])[1,1]
            eta[j,r] = rand(Uniform(nxi[j,r],1), 1)[1,1]
            nu[j,r] = norminvcdf(eta[j,r])
            P[j,r] =  1 - cdf(Normal(), xi[j,r])
        end
    end
    # Containter for the product of draws
    H = zeros(1,R)
        for r in 1:R
            H[r] = prod(P[:,r])
        end
    # Average of the products
    I_ghk = (1/R)*sum(H)
end

GHK_Sim (generic function with 1 method)

In [5]:
function crude1(R::Int64)
    # This function is a non-general Crude Freq simulator for the above probability.
    # This function takes only the number of simulations as the argument.
    # Random Seed
    srand(757537);
    # Draw Uniform Variables
    u = rand(d, R)
    # Get Standard Normal Draws from Uniform, note vectorization
    eta = norminvcdf.(u)
    # Get the Values Greater than the Lower Bound
    LB = (-1)*inv(G)*mu' * ones(1,R)
    nu = (eta.>=LB)
    nu = convert(Array{Int64,2},nu)
    # Containter for the product of draws
    H = zeros(1,R)
    for r in 1:R
        H[r] = prod(nu[:,r])
    end
    # Average of the products
    I_cf = (1/R)*sum(H)
end

crude1 (generic function with 1 method)

In [6]:
function crudeAVS(R::Int64)
    # This function is a non-general Crude Freq Antivariate simulator for the above probability.
    # This function takes only the number of simulations as the argument.    
    # Random Seed
    srand(757537);
    # Draw Normal(0,1) Variables, note the negative correlated variables, neg
    eta = rand(d, R)
    neg = 1 - eta
    # Get Standard Normal Draws from Uniform, note vectorization
    eta = norminvcdf.(eta)
    neg = norminvcdf.(neg)    
    # Get the Values Greater than the Lower Bound
    LB = (-1)*inv(G)*mu' * ones(1,R)
    nu1 = (eta.>=LB)
    nu1 = convert(Array{Int64,2},nu1)
    nu2 = (neg.>=LB)
    nu2 = convert(Array{Int64,2},nu2)    
    # Containter for the product of draws
    H1 = zeros(1,R)
    for r in 1:R
        H1[r] = prod(nu1[:,r])
    end
    H2 = zeros(1,R)
    for r in 1:R
        H2[r] = prod(nu2[:,r])
    end    
    # Average of the products
    I_cfavs = 0.5*((1/R)*sum(H1) + (1/R)*sum(H2))
end

crudeAVS (generic function with 1 method)

In [7]:
# This runs the three simulator against each other for various simulations.
for n in [25, 100,500,1000,2000]
    ghk = round(GHK_Sim(n),4)
    cf1 = round(crude1(n),4)
    cfavs = round(crudeAVS(n),4)
    println("GHK= $ghk, CrdFrq =$cf1, CrdAVS = $cfavs for $n")
end

GHK= 0.0892, CrdFrq =0.04, CrdAVS = 0.02 for 25
GHK= 0.0842, CrdFrq =0.04, CrdAVS = 0.025 for 100
GHK= 0.0834, CrdFrq =0.042, CrdAVS = 0.035 for 500
GHK= 0.0831, CrdFrq =0.045, CrdAVS = 0.0405 for 1000
GHK= 0.0834, CrdFrq =0.0445, CrdAVS = 0.041 for 2000


This section assumes we do not know the first two values of $\mu$, $(\mu_1,\mu_2)$, but still know $(\mu_3, \mu_4, \mu_5)$ and $\Omega$.

We want to use the GHK simulator to search for the Simulated MLE for the parameters, $\theta := (\mu_1,\mu_2)$, assuming we only observe $D_i = \mathcal{1}(Y_i \ge 0)$.

Like before, we note that $Y_i = \mu + \Gamma^{\prime}X_i$, for $X\sim \mathcal{N}(0,\mathcal{I}_d)$.
Thus, $D_i = 1 \iff X_i \ge -\Gamma^{-1}\mu$.

Therefore, we can get $\mathcal{l}_i(\theta) = D_i \ln[\Phi(X_i + \Gamma^{-1}\mu;\ \theta)] + (1 - D_i) \ln[1 - \Phi(X_i + \Gamma^{-1}\mu;\ \theta)]$.

In [8]:
# Set the observations of the sample and the simulations
N = 1000;
R = 1000;
# Draw random variable for generating Y
# Note we are creating the Y variable in the same way we might estimate it!
srand(757537)
eta = rand(Normal(), d, N);
Y = zeros(d,N);
for i in 1:N
    Y[:,i] = mu' + G*eta[:,i]
end
# Create the binary D variable we observe
temp = (Y.>=0);
temp = convert(Array{Float64,2},temp);
D = zeros(1,N);
for i in 1:N
    D[i] = prod(temp[:,i])
end
print("The number of Positive Observations are $(sum(D)) out of $N and gives an empirical probability of $(sum(D)/N)")

The number of Positive Observations are 82.0 out of 1000 and gives an empirical probability of 0.082

In [9]:
function GHK_MUer(mu::Vector)
    # This function is a non-general GHK simulator for the above probability.
    # This function takes only the number of simulations as the argument.
    # Random Seed
    srand(757537);
    # Radnom Draws
    # Containers
    xi = zeros(d,R);
    nxi = zeros(d,R);    
    eta = zeros(d,R);
    nu = zeros(d,R);
    P = zeros(d,R);
    x1 = cdf(Normal(),((-1*mu[1]) / G[1,1]))
    eta[1,:] = rand(Uniform(x1,1), R)
    nu[1,:] = norminvcdf.(eta[1,:])
    P[1,:] =  (1 - x1)*ones(1,R)
    for r = 1:R
        for j = 2:d
            xi[j,r] = ((-1)*(mu[j] + (G[j,1:(j-1)]'*nu[1:(j-1),r])) / G[j,j])[1,1]
            nxi[j,r] = cdf(Normal(), xi[j,r])[1,1]
            eta[j,r] = rand(Uniform(nxi[j,r],1), 1)[1,1]
            nu[j,r] = norminvcdf(eta[j,r])
            P[j,r] =  1 - cdf(Normal(), xi[j,r])
        end
    end
    # Containter for the product of draws
    H = zeros(1,R)
        for r in 1:R
            H[r] = prod(P[:,r])
        end
    # Average of the products
    I_ghk = (1/R)*sum(H)
end

GHK_MUer (generic function with 1 method)

In [10]:
function loglike_MU(TH)
    # This function creates the loglikelihood for the simulated MLE 
    # Preallocate
    simD = ones(N)
    lnsimD = Array(Float64,N,1)
    mlnsimD = Array(Float64,N,1)
    lnL = Array(Float64,N,1)
    # Math
    mu0 = vec([TH mu[2:d]'])
    simD = simD * GHK_MUer(mu0)
    lnsimD = log(simD)
    mlnsimD = log(1-simD)
    lnL = ((-1)*(1/N)*sum(D.*lnsimD + (1-D).*mlnsimD))
    return(lnL[1,1])
end

loglike_MU (generic function with 1 method)

In [11]:
# This used the NM optimization routine to search for the \theta that minimizes(-lnL)
# Note I am stacking the deck by giving the initial values the true sign.
res = optimize(loglike_MU, -0.75, 0.75)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-0.750000, 0.750000]
 * Minimizer: 4.555521e-01
 * Minimum: 2.836271e+02
 * Iterations: 23
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 24

In [12]:
round(Optim.minimizer(res),3)

0.456